# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Actividad Semana 9

### **Modelos Transformer & LLM**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Samuel Emanuel Aguilar Garcia - A00816565
*   Luis Alejandro Aguilar Díaz - A01795362
*   Fernando Acevedo Ortegate - A01794022
*   Jorge Antonio Acosta Rosales - A01381552


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm.notebook import tqdm
from huggingface_hub import notebook_login
import random

##**Pregunta - 1:**



Descarga el archivo **amazonbaby5000.csv** que se encuentra en Canvas. Este archivo está formado de 5 mil comentarios en inglés sobre productos para bebé adquiridos a través de la plataforma de Amazon.



In [ ]:
# Establecer la ruta del archivo
path = '/content/drive/MyDrive/NLP/amazonbaby5000.csv'

# Abrir archivo en un dataframe
df = pd.read_csv(path)

# Desplegar las primeras columnas del archivo para visualizar que se cargó correctamente
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  5000 non-null   object
 1   rating  5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.2+ KB
None
                                              review  rating
0  I think it is more Expensive than drugstore th...       0
1  When I saw this on Amazon, I put it into my wi...       1
2  We really like these valances.  They have such...       1
3  No light emits from the night light. They pain...       0
4  I was really hoping for this to be a conventie...       0


Se confirma que nuestro dataframe con la información del csv tiene 5000 comentarios.

##**Pregunta - 2:**

Realiza una partición de los datos en el porcentaje que consideres adecuado, en entrenamiento y prueba.

In [ ]:
# Separamos los datos de los comentarios (review), de las calificaciones (rating)
X = df['review']
y = df['rating']

# Se hace la partición de los datos 70/30 entre el entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=69)

In [ ]:
# Confirmamos la partición la partición de los datos
print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]}')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]}')

Tamaño del conjunto de entrenamiento: 3500
Tamaño del conjunto de prueba: 1500


#**Pregunta - 3:**


Utiliza un modelo Transformer de HuggingFace para análisis de sentimiento en inglés y lleva a
cabo la predicción de los comentarios en los siguientes casos:

a) Utiliza uno de los modelos Transformer pre-entrenados que consideres adecuado de
DistilBERT de Huggingface para análisis de sentimiento y lleva a cabo el entrenamiento y evaluación del desempeño con exactitud (accuracy) y matriz de confusión.

NOTA:
Recuerda obtener el porcentaje de las clases positivas y negativas para tener una idea clara de si el modelo encontrado queda subentrenado.

In [ ]:
# Desplegar porcentaje de clases positivas y negativas
clases = np.bincount(y_train)
total = len(y_train)
print(f"Porcentaje de clase Negativa: {(clases[0] / total) * 100:.2f}%")
print(f"Porcentaje de clase Positiva: {(clases[1] / total) * 100:.2f}%")

Porcentaje de clase Negativa: 23.80%
Porcentaje de clase Positiva: 76.20%


Este resultado nos muestra que las clases estan muy desbalancedas y que en el conjunto de entrenamiento existen una mayor cantidad de ejemplos con una clasificación positiva. Podemos confirmar revisando la distribución de las clases en el conjunto total, para descartar un tema del shuffle en la partición.

In [ ]:
# Desplegar porcentaje de clases positivas y negativas totales
clases = np.bincount(y)
total = len(y)
print(f"Porcentaje de clase Negativa: {(clases[0] / total) * 100:.2f}%")
print(f"Porcentaje de clase Positiva: {(clases[1] / total) * 100:.2f}%")

Porcentaje de clase Negativa: 23.20%
Porcentaje de clase Positiva: 76.80%


Observamos que esta distribución se encuentra así en el conjunto total, por lo que hay que tomarlo en cuenta para evitar un subentrenamiento en alguna clase en específico.

In [ ]:
# Cargar el modelo
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Función para accuracy

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
    }

In [ ]:
# Preparar los comentarios para usarlos en el modelo
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=512)

train_labels = torch.tensor(y_train.tolist())
test_labels = torch.tensor(y_test.tolist())


In [ ]:
# Crear un Dataset a partir de las codificaciones y etiquetas
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, train_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

In [ ]:
# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda


In [ ]:
# Crear un Trainer con los valores predeterminados
model_test = model
trainer = Trainer(
   model=model_test,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

In [ ]:
model_device = next(model.parameters()).device
print(f"Dispositivo del modelo: {model_device}")

Dispositivo del modelo: cuda:0


In [ ]:
# Entrenar el modelo
trainer.train()

Step,Training Loss
500,0.336600
1000,0.128300


TrainOutput(global_step=1314, training_loss=0.18859221401824255, metrics={'train_runtime': 492.8001, 'train_samples_per_second': 21.307, 'train_steps_per_second': 2.666, 'total_flos': 1390907685888000.0, 'train_loss': 0.18859221401824255, 'epoch': 3.0})

Se realizará el análisis de manera iterativa para disminuir el impacto generado en la RAM.

In [ ]:
# Función para aplicar el modelo a los valores seleccionados, realizar predicciones y calcular accuracy y la matriz de confusión
def evaluate_model(model, val_encodings, val_labels, batch_size=32):
    # Listas para almacenar las predicciones y las etiquetas verdaderas
    predictions = []
    true_labels = []

    # Definir el dispositivo (GPU o CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Asegurarse de que los encodings están en formato tensor
    val_encodings = {k: torch.tensor(v) for k, v in val_encodings.items()}

    # Crear lotes y realizar predicciones sobre los datos de validación con una barra de progreso
    for batch_start in tqdm(range(0, len(val_encodings['input_ids']), batch_size), desc="Evaluating"):
        batch_end = min(batch_start + batch_size, len(val_encodings['input_ids']))

        # Crear un diccionario con los tensores del lote actual
        batch_encodings = {k: v[batch_start:batch_end].to(device) for k, v in val_encodings.items()}

        # Generar las predicciones en el lote actual
        with torch.no_grad():
            outputs = model(**batch_encodings)

        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=1).tolist()

        # Añadir predicciones y etiquetas verdaderas al conjunto total
        predictions.extend(batch_predictions)
        true_labels.extend(val_labels[batch_start:batch_end].tolist())

    # Calcular la exactitud
    accuracy = accuracy_score(true_labels, predictions)
    print(f'Accuracy: {accuracy:.4f}')

    # Calcular y mostrar la matriz de confusión
    cm = confusion_matrix(true_labels, predictions)
    print('Confusion Matrix:')
    print(cm)

In [ ]:
evaluate_model(model_test, test_encodings, test_labels, batch_size=32)

Evaluating:   0%|          | 0/47 [00:00<?, ?it/s]

Accuracy: 0.9120
Confusion Matrix:
[[ 252   75]
 [  57 1116]]


El accuracy en los datos es bastante alto, lo cual indica un entrenamiento bastante acertado. Como se esperaba, para los comentarios negativos al tener una menor cantidad tiene un mayor porcentaje de error.

b) Realiza una partición del conjunto de entrenamiento en uno nuevo de entrenamiento y validación, con los porcentajes que consideres adecuados y utilizando el mismo modelo DistilBERT del inciso anterior, ajusta sus hiperparámetros (fine-tuning) para entrenar y evaluar el desempeño de este nuevo modelo.


In [ ]:
# Se hace la partición de los datos 70/30 entre del entrenamiento en entrenamiento y validación.
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=True, random_state=69)

In [ ]:
# Confirmamos la partición de los datos
print(f'Tamaño del conjunto de entrenamiento: {X_train2.shape[0]}')
print(f'Tamaño del conjunto de validación: {X_val.shape[0]}')

Tamaño del conjunto de entrenamiento: 2450
Tamaño del conjunto de validación: 1050


In [ ]:
# Preparar los comentarios para usarlos en el modelo
train2_encodings = tokenizer(X_train2.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=512)

train2_labels = torch.tensor(y_train2.tolist())
val_labels = torch.tensor(y_val.tolist())

In [ ]:
train2_dataset = ReviewDataset(train2_encodings, train2_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

In [ ]:
notebook_login()   # genera un token que sea de escritura (write):

In [ ]:
# Definir los argumentos para el fine tunning
repo_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

In [ ]:
# Crear un Trainer con los valores predeterminados
model_FineTune = model
trainer2 = Trainer(
   model=model_FineTune,
   args=training_args,
   train_dataset=train2_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

In [ ]:
evaluate_model(model_FineTune, val_encodings, val_labels, batch_size=32)

Evaluating:   0%|          | 0/33 [00:00<?, ?it/s]

Accuracy: 0.9933
Confusion Matrix:
[[254   5]
 [  2 789]]


c) Con el mejor de los dos modelos anteriores, evalúa el desempeño utilizando ahora el conjunto de prueba.

In [ ]:
evaluate_model(model_FineTune, test_encodings, test_labels, batch_size=32)

Evaluating:   0%|          | 0/47 [00:00<?, ?it/s]

Accuracy: 0.9133
Confusion Matrix:
[[ 256   71]
 [  59 1114]]


d) Incluye tus comentarios.

Observamos que el resultado después de el fine tunning en los valores de validación es bastante alto. No obstante al momento de evaluar los datos de prueba el resultado es ligeramente mejor al entrenamiento normal.

#**Pregunta - 4:**

En este ejercicio vas a realizar una comparación simple entre un modelo Transformer preentrenado para traducción de inglés a español, con un modelo LLM (Large Language Model).


Para ello selecciona de manera aleatoria 10 comentarios del conjunto de AmazonBaby5000.

In [ ]:
# Fijar semilla
random.seed(69)

# Crear una lista solo con las reseñas
reviews = X.tolist()

# Escoger 10 oraciones de manera aleatoria
random_reviews = random.sample(reviews, 10)

# Imprimir las 10 oraciones escogidas aleatoriamente
for idx, review in enumerate(random_reviews, start=1):
    print(f"Comentario {idx}: {review}\n")

Comentario 1: I so wanted to love this travel crib, but I just can't. After copious amount of research I purchased the Lotus (at the hefty $200 price point) for a trip to visit family alone with my 8-month-old that involved flying. This thing seemed like the perfect solution since it was compact enough to fit in an overhead storage bin. The compact size is the ONLY thing I ended up liking about it. Below are my major disappointments and reasons I am selling it after about 4 months of use:- velcro straps take forever to thread through the base to secure the mattress and sheet (all while hunched over at a very uncomfortable angle since you have to have one arm in and one are out of the crib to do it).- the mattress DOES NOT LIE FLAT once the sheet is over it. It pulls the mattress to a point that baby has to lie on it (uncomfortably I'm sure) to flatten it out, and even then it still pulls up on the ends. I have no idea how they got it to lie flat with the sheet on it in the product pict

a) Utiliza el modelo Tansformer pre-entrenado para traducción de inglés a español
llamado Helsinki-NLP/opus-mt-en-es de Huggingface y traduce los 10 comentarios que seleccionaste.


In [ ]:
# Cargar el modelo de traducción
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_trad = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model_trad = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# Inicializar lista de traducciones
translate_Helsinki = []

# Iterar por cada comentario
for idx, review in enumerate(random_reviews, start=1):

    # Tokenizar el comentario
    inputs = tokenizer_trad(review, return_tensors="pt")

    # Realizar la traducción
    with tokenizer_trad.as_target_tokenizer():
        outputs = model_trad.generate(**inputs)

    # Decodificar la traducción
    translated_text = tokenizer_trad.decode(outputs[0], skip_special_tokens=True)
    translate_Helsinki.append(translated_text)

    print(f"Comentario {idx} (Inglés): {review}")
    print(f"Comentario {idx} (Español): {translated_text}\n")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Comentario 1 (Inglés): I so wanted to love this travel crib, but I just can't. After copious amount of research I purchased the Lotus (at the hefty $200 price point) for a trip to visit family alone with my 8-month-old that involved flying. This thing seemed like the perfect solution since it was compact enough to fit in an overhead storage bin. The compact size is the ONLY thing I ended up liking about it. Below are my major disappointments and reasons I am selling it after about 4 months of use:- velcro straps take forever to thread through the base to secure the mattress and sheet (all while hunched over at a very uncomfortable angle since you have to have one arm in and one are out of the crib to do it).- the mattress DOES NOT LIE FLAT once the sheet is over it. It pulls the mattress to a point that baby has to lie on it (uncomfortably I'm sure) to flatten it out, and even then it still pulls up on the ends. I have no idea how they got it to lie flat with the sheet on it in the pro

Utiliza ahora el LLM Gemini de Google a través de su API para traducir los mismos 10 comentarios del inciso anterior.



*   NOTA: deberás proponer el prompt que consideres adecuado para la traducción, incluyendo si consideras que ayuda, que tome en cuenta errores tipográficos (typos), o algún otro tipo de consideración.
*   NOTA: Puedes consultar la siguiente liga para familiarizarte con la API de Gemini:
https://ai.google.dev/gemini-api/docs/get-started/tutorial?hl=es-419&lang=python






In [ ]:
pip install -q -U google-generativeai

In [ ]:
# Importar paquetes necesarios
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
# Selección de llave API
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Enumeración de modelos disponibles
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
# Importar el modelo Gemini 1.5
model_Gemini = genai.GenerativeModel('gemini-1.5-flash')

# Función para traducir el texto
def translate_GeminiLLM (text):
    translate = model_Gemini.generate_content(
        #Prompt usado para pedir la traducción
        ['Translate the following text form English to Spanish, take into account any typographical error and that it is a review of a product: ', text]
    )

    translated_GText = translate.text
    return translated_GText

In [ ]:
# Inicialización de lista con los comentarios traducidos
translated_Gemini = []

# Iterar sobre los comentarios y traducir cada uno
for review in random_reviews:
    translated_review = translate_GeminiLLM(review)
    translated_Gemini.append(translated_review)

# Imprimir los comentarios traducidos
for idx, (original, translated) in enumerate(zip(random_reviews, translated_Gemini), start=1):
    print(f"Comentario {idx} (Inglés): {original}")
    print(f"Comentario {idx} (Español): {translated}\n")


Comentario 1 (Inglés): I so wanted to love this travel crib, but I just can't. After copious amount of research I purchased the Lotus (at the hefty $200 price point) for a trip to visit family alone with my 8-month-old that involved flying. This thing seemed like the perfect solution since it was compact enough to fit in an overhead storage bin. The compact size is the ONLY thing I ended up liking about it. Below are my major disappointments and reasons I am selling it after about 4 months of use:- velcro straps take forever to thread through the base to secure the mattress and sheet (all while hunched over at a very uncomfortable angle since you have to have one arm in and one are out of the crib to do it).- the mattress DOES NOT LIE FLAT once the sheet is over it. It pulls the mattress to a point that baby has to lie on it (uncomfortably I'm sure) to flatten it out, and even then it still pulls up on the ends. I have no idea how they got it to lie flat with the sheet on it in the pro

c)  Despliega ambas traducciones de manera tabular y compara los resultados obtenidos.


Incluye tus conclusiones y los pros y contras que detectas en cada técnica.
NOTA: Aunque la comparación será de manera subjetiva, el punto de vista de una
persona es muy valioso y nos puede proporcionar una buena idea de los desempeños de los modelos.

In [ ]:
# Pasar toda los datos originales y las traducciones a un DataFrame
df_trad = pd.DataFrame({
    'Original': random_reviews,
    'Transformer Helsinki-NLP': translate_Helsinki,
    'LLM Gemini': translated_Gemini
})

In [ ]:
from IPython.display import display

In [ ]:
# Desplegar los comentarios originales y las traducciones en una tabla
display(df_trad)

,Original,Transformer Helsinki-NLP,LLM Gemini
0,"I so wanted to love this travel crib, but I ju...",Después de una gran cantidad de investigación ...,"Quería amar tanto esta cuna de viaje, pero sim..."
1,Wanted a digital caliper. They are pretty expe...,Quería un calibrador digital. Son bastante car...,Quería un calibre digital. Son bastante caros ...
2,Adding the extension set seemed to almost doub...,Añadir el conjunto de extensión parecía casi d...,La adición del juego de extensión pareció casi...
3,I couldn't resist the affordable price and was...,No pude resistirme al precio asequible y estab...,¡No pude resistirme al precio accesible y esta...
4,Absolutely love this seat protector. The entir...,Absolutamente me encanta este protector de asi...,¡Me encanta este protector de asiento! Toda la...
5,Love this Minky Dot material. Its super soft f...,Me encanta este material de punto Minky. Su sú...,¡Me encanta este material Minky Dot! Es súper ...
6,The base stopped working. The main camera soc...,La base dejó de funcionar. El enchufe principa...,La base dejó de funcionar. El enchufe de la cá...
7,My five month old daughter enjoys this toy imm...,Mi hija de cinco meses disfruta de este juguet...,A mi hija de cinco meses le encanta este jugue...
8,Cleaning my son with warm wipes was something ...,Limpiar a mi hijo con toallitas calientes era ...,Limpiar a mi hijo con toallitas tibias era alg...
9,It leaves a film on your window after the firs...,Deja una película en la ventana después del pr...,Deja una película en la ventana después del pr...


Ambas traducciones reflejan interpretaciones correctas de la oración original al español, a menudo mostrando resultados iguales o muy similares. Sin embargo, la traducción realizada con el LLM de Gemini puede considerarse superior en los ejemplos presentados. Este modelo proporciona una representación más fiel al texto original, especialmente en términos de evitar traducciones demasiado literales y capturar el contexto de la oración.

Por ejemplo, en la primera oración, el transformador Helsinki omite la primera parte, mientras que el LLM de Gemini la incluye en la traducción. Además, en la oración 8, al traducir "warm wipes" como "toallitas tibias" en lugar de "calientes," demuestra una mejor comprensión del contexto, ya que aunque "warm" se traduce más literalmente como "caliente," en este caso "tibias" es más adecuado.

Esto es aún más evidente en oraciones más largas, donde la interpretación del LLM de Gemini resulta ser más precisa y contextualizada.

Incluye tus conclusiones finales de la actividad.


En esta actividad se reforzaron los conocimientos adquiridos durante la semana sobre los transformers. En particular, se revisó de manera práctica cómo usar modelos preentrenados para generar resultados más rápidos. A veces, es posible necesitar un modelo sin disponer de los datos necesarios para realizar un entrenamiento óptimo. En estos casos, se pueden utilizar modelos preentrenados y ajustar su entrenamiento con la información disponible para hacerlos más precisos para nuestro uso. Estos transformers son útiles para diversas tareas; en esta actividad, los empleamos para la clasificación y traducción de textos. Además, existen muchas otras aplicaciones, como la generación de texto y el análisis de imágenes.

Otra opción para la traducción son los LLM (Large Language Models). En esta actividad, se utilizó el LLM de Gemini, que proporcionó resultados excelentes, considerando el contexto de las oraciones para elegir las palabras adecuadas. Aunque los resultados entre ambas técnicas de traducción fueron muy similares, la elección de una u otra depende de los recursos disponibles.

##**Fin de la Actividad de Modelos Transformer & LLM**